# Precip Compare: Wet Day Spell Length, 1981-2011

Compare the wet day spell length distributions and samples among the three data sets for the period 1981-2011.

This climate period has three data sets

1. PRISM (actual data)
2. LOCA (downscaled GCM results)
3. BCCA (downscaled GCM results)

Negative binomial distributions have been fit to all of these data sets using R in separate Jupyter notebooks. The purpose of this notebook is to compare the fitted distributions and samples to see how different they are from each other.

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats as sstats

Parameters, primarily input file names

In [3]:
PRISM_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Rech' \
           r'arge\Data\JNotes\Processed\PRISM\WetDays_1981-2010.pickle'
JAN_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Jan_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
FEB_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Feb_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
MAR_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Mar_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
APR_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Apr_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
MAY_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\May_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
JUN_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Jun_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
JUL_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Jul_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
AUG_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Aug_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
SEP_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Sep_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
OCT_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Oct_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
NOV_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Nov_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'
DEC_CMIP5_IN = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Re' \
               r'charge\Data\JNotes\Processed\CMIP5\CMIP5_1981_WetDays\Dec_Wet' \
               r'Cnt_CMIP5_1981-2010.pickle'

In [4]:
CMIP5_XLSX = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Data\JN' \
             r'otes\Processed\CMIP5\CMIP5_1981_WetDays\CMIP5_Wet_RSummary_1981-2010.xlsx'
LOCA_SHT = r'LOCA_Dist_Properties'
BCCA_SHT = r'BCCA_Dist_Properties'
PRISM_XLSX = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Data\JN' \
             r'otes\Processed\PRISM\PRISM_WetDays\PRISM_Wet_RSummary.xlsx'
PRISM_SHT = r'Dist_Properties'
HDRS = [ "Month", #0
         "Sample Mean (mu)", #1
         "Sample Size (r)", #2
         "Sample Prob (p)", #3
         "FitDist Mean (mu)", #4
         "FitDist Size (r)", #5
         "p-value", #6
]

In [5]:
OUT_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Data\JN' \
          r'otes\Processed\Precip_Compare_1981-2010'

In [6]:
LOCA_GRID_END = 168       # the last LOCA grid cell

## Load Files

Start with the spreadsheets that have the fitted distribution parameters for the negative binomial distribution

In [7]:
PRDistDF = pd.read_excel( PRISM_XLSX, sheet_name='Dist_Properties', header=None,
                          index_col=0, names=HDRS, skiprows=[0] )

In [8]:
display( HTML( PRDistDF.head().to_html() ) )

,Month,Sample Mean (mu),Sample Size (r),Sample Prob (p),FitDist Mean (mu),FitDist Size (r),p-value
1,1,1.866392,3.483419,1.000000,1.866322,6.860232e+06,0.000000e+00
2,2,1.815739,3.296908,0.999999,1.815874,1.958205e+06,7.098435e-144
3,3,1.761789,3.103899,1.000000,1.761965,8.392017e+06,6.405878e-315
4,4,1.768595,3.127928,0.999999,1.768731,1.958205e+06,1.187500e-145
5,5,1.862177,3.467703,0.999993,1.862747,2.810599e+05,0.000000e+00


In [9]:
LODistDF = pd.read_excel( CMIP5_XLSX, sheet_name=LOCA_SHT, header=None,
                          index_col=0, names=HDRS, skiprows=[0] )

In [10]:
display( HTML( LODistDF.head().to_html() ) )

,Month,Sample Mean (mu),Sample Size (r),Sample Prob (p),FitDist Mean (mu),FitDist Size (r),p-value
1,1,1.862078,7.215064,0.999993,1.862262,2.535299e+05,0
2,2,1.986150,11.283718,1.000000,1.986097,8.404404e+06,0
3,3,1.894751,12.939019,0.999999,1.894854,3.416294e+06,0
4,4,2.301874,5.963750,0.819556,2.301808,1.045453e+01,0
5,5,2.692112,3.264539,0.663506,2.691926,5.307985e+00,0


In [11]:
BCDistDF = pd.read_excel( CMIP5_XLSX, sheet_name=BCCA_SHT, header=None,
                          index_col=0, names=HDRS, skiprows=[0] )

In [12]:
display( HTML( BCDistDF.head().to_html() ) )

,Month,Sample Mean (mu),Sample Size (r),Sample Prob (p),FitDist Mean (mu),FitDist Size (r),p-value
1,1,1.993002,79.414175,0.999994,1.992670,309044.207584,0
2,2,2.036637,65.707221,0.999993,2.036861,274341.085422,0
3,3,2.026520,27.861059,0.960882,2.026625,49.780868,0
4,4,2.462707,2.160967,0.636479,2.462660,4.311802,0
5,5,3.192043,1.457214,0.442022,3.191634,2.528366,0


In [13]:
HDRS.append( "FitDist Prob (p)" ) # 7
HDRS.append( "FitDist Var" ) #8
HDRS.append( "FitDist Std" ) #9
HDRS.append( "FitDist AltForm P" ) #10
HDRS.append( "FitDist AltForm N" ) #11

In [14]:
PRDistDF[HDRS[7]] = ( PRDistDF[HDRS[2]] / ( PRDistDF[HDRS[2]] + PRDistDF[HDRS[1]] ))
PRDistDF[HDRS[8]] = ( PRDistDF[HDRS[1]] + ( ( 1.0 / PRDistDF[HDRS[2]]) * (PRDistDF[HDRS[1]]**2.0) ) )
PRDistDF[HDRS[9]] = np.sqrt( np.array( PRDistDF[HDRS[8]], dtype=np.float32 ) )
PRDistDF[HDRS[10]] = 1.0 - ((PRDistDF[HDRS[8]] - PRDistDF[HDRS[4]]) / PRDistDF[HDRS[8]] )
PRDistDF[HDRS[11]] = ( PRDistDF[HDRS[4]]**2.0 )/(PRDistDF[HDRS[8]] - PRDistDF[HDRS[4]])

In [15]:
display( HTML( PRDistDF.to_html() ) )

,Month,Sample Mean (mu),Sample Size (r),Sample Prob (p),FitDist Mean (mu),FitDist Size (r),p-value,FitDist Prob (p),FitDist Var,FitDist Std,FitDist AltForm P,FitDist AltForm N
1,1,1.866392,3.483419,1.000000,1.866322,6.860232e+06,0.000000e+00,0.651129,2.866392,1.693042,0.651105,3.482914
2,2,1.815739,3.296908,0.999999,1.815874,1.958205e+06,7.098435e-144,0.644853,2.815739,1.678016,0.644902,3.297847
3,3,1.761789,3.103899,1.000000,1.761965,8.392017e+06,6.405878e-315,0.637916,2.761789,1.661863,0.637980,3.105070
4,4,1.768595,3.127928,0.999999,1.768731,1.958205e+06,1.187500e-145,0.638806,2.768595,1.663910,0.638855,3.128831
5,5,1.862177,3.467703,0.999993,1.862747,2.810599e+05,0.000000e+00,0.650616,2.862177,1.691797,0.650815,3.471808
6,6,2.071890,4.292729,0.881858,2.071984,1.546611e+01,0.000000e+00,0.674468,3.071890,1.752681,0.674498,4.293524
7,7,2.056709,4.230053,0.952332,2.056711,4.109010e+01,6.853384e-315,0.672851,3.056709,1.748345,0.672851,4.230068
8,8,2.098422,4.403374,0.879597,2.098580,1.533100e+01,0.000000e+00,0.677255,3.098422,1.760233,0.677306,4.404735
9,9,1.863289,3.471844,1.000000,1.863442,4.317364e+06,3.880052e-218,0.650751,2.863289,1.692125,0.650805,3.472946
10,10,2.170252,3.596253,0.784404,2.170282,7.896168e+00,0.000000e+00,0.623645,3.479947,1.865462,0.623654,3.596434


In [16]:
LODistDF[HDRS[7]] = ( LODistDF[HDRS[2]] / ( LODistDF[HDRS[2]] + LODistDF[HDRS[1]] ))
LODistDF[HDRS[8]] = ( LODistDF[HDRS[1]] + ( ( 1.0 / LODistDF[HDRS[2]]) * (LODistDF[HDRS[1]]**2.0) ) )
LODistDF[HDRS[9]] = np.sqrt( np.array( LODistDF[HDRS[8]], dtype=np.float32 ) )
LODistDF[HDRS[10]] = 1.0 - ((LODistDF[HDRS[8]] - LODistDF[HDRS[4]]) / LODistDF[HDRS[8]] )
LODistDF[HDRS[11]] = ( LODistDF[HDRS[4]]**2.0 )/(LODistDF[HDRS[8]] - LODistDF[HDRS[4]])

In [17]:
display( HTML( LODistDF.to_html() ) )

,Month,Sample Mean (mu),Sample Size (r),Sample Prob (p),FitDist Mean (mu),FitDist Size (r),p-value,FitDist Prob (p),FitDist Var,FitDist Std,FitDist AltForm P,FitDist AltForm N
1,1,1.862078,7.215064,0.999993,1.862262,2.535299e+05,0,0.794861,2.342647,1.530571,0.794939,7.219238
2,2,1.986150,11.283718,1.000000,1.986097,8.404404e+06,0,0.850326,2.335750,1.528316,0.850304,11.281422
3,3,1.894751,12.939019,0.999999,1.894854,3.416294e+06,0,0.872268,2.172213,1.473843,0.872315,12.945242
4,4,2.301874,5.963750,0.819556,2.301808,1.045453e+01,0,0.721512,3.190345,1.786154,0.721492,5.962974
5,5,2.692112,3.264539,0.663506,2.691926,5.307985e+00,0,0.548049,4.912171,2.216342,0.548011,3.263812
6,6,2.915002,2.580594,0.587767,2.915073,4.156345e+00,0,0.469575,6.207747,2.491535,0.469586,2.580774
7,7,2.626201,2.607539,0.628176,2.626236,4.436880e+00,0,0.498217,5.271199,2.295909,0.498224,2.607641
8,8,2.621325,3.018100,0.652720,2.620971,4.926165e+00,0,0.535179,4.898037,2.213151,0.535106,3.016817
9,9,2.929591,2.758973,0.590071,2.929320,4.216608e+00,0,0.485003,6.040351,2.457712,0.484959,2.758223
10,10,2.497473,6.716813,0.803050,2.497329,1.018268e+01,0,0.728956,3.426094,1.850971,0.728914,6.715000


In [18]:
BCDistDF[HDRS[7]] = ( BCDistDF[HDRS[2]] / ( BCDistDF[HDRS[2]] + BCDistDF[HDRS[1]] ))
BCDistDF[HDRS[8]] = ( BCDistDF[HDRS[1]] + ( ( 1.0 / BCDistDF[HDRS[2]]) * (BCDistDF[HDRS[1]]**2.0) ) )
BCDistDF[HDRS[9]] = np.sqrt( np.array( BCDistDF[HDRS[8]], dtype=np.float32 ) )
BCDistDF[HDRS[10]] = 1.0 - ((BCDistDF[HDRS[8]] - BCDistDF[HDRS[4]]) / BCDistDF[HDRS[8]] )
BCDistDF[HDRS[11]] = ( BCDistDF[HDRS[4]]**2.0 )/(BCDistDF[HDRS[8]] - BCDistDF[HDRS[4]])

In [19]:
display( HTML( BCDistDF.to_html() ) )

,Month,Sample Mean (mu),Sample Size (r),Sample Prob (p),FitDist Mean (mu),FitDist Size (r),p-value,FitDist Prob (p),FitDist Var,FitDist Std,FitDist AltForm P,FitDist AltForm N
1,1,1.993002,79.414175,0.999994,1.992670,309044.207584,0,0.975518,2.043019,1.429342,0.975356,78.864284
2,2,2.036637,65.707221,0.999993,2.036861,274341.085422,0,0.969936,2.099764,1.449056,0.970043,65.955137
3,3,2.026520,27.861059,0.960882,2.026625,49.780868,0,0.932195,2.173922,1.474423,0.932244,27.883844
4,4,2.462707,2.160967,0.636479,2.462660,4.311802,0,0.467370,5.269287,2.295493,0.467361,2.160847
5,5,3.192043,1.457214,0.442022,3.191634,2.528366,0,0.313429,10.184247,3.191277,0.313389,1.456755
6,6,2.874320,1.611180,0.494438,2.874571,2.811326,0,0.359197,8.002063,2.828792,0.359229,1.611540
7,7,2.669911,1.509936,0.523023,2.669843,2.927585,0,0.361242,7.390922,2.718625,0.361233,1.509837
8,8,2.879676,1.514489,0.484519,2.879949,2.706971,0,0.344659,8.355140,2.890526,0.344692,1.514853
9,9,3.055407,1.918849,0.493256,3.055099,2.973783,0,0.385756,7.920571,2.814351,0.385717,1.918341
10,10,2.488294,3.224386,0.681936,2.488041,5.334425,0,0.564426,4.408539,2.099652,0.564369,3.223304


In [21]:
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "WetDist_Compare_1981-2011.xlsx" ) )
with pd.ExcelWriter( OutFiler ) as writer:
    PRDistDF.to_excel( writer, sheet_name="PRISM", na_rep=str(np.nan), columns=HDRS,
                       index=False )
    LODistDF.to_excel( writer, sheet_name="LOCA", na_rep=str(np.nan), columns=HDRS,
                       index=False )
    BCDistDF.to_excel( writer, sheet_name="BCCA", na_rep=str(np.nan), columns=HDRS,
                       index=False )

### Load and Reduce DataFrames

In [ ]:
PrismDF = pd.read_pickle( PRISM_IN )

In [ ]:
display( HTML( PrismDF.head().to_html() ) )

In [ ]:
JanPRDF = PrismDF[PrismDF["Month"] == 1].copy()
FebPRDF = PrismDF[PrismDF["Month"] == 2].copy()
MarPRDF = PrismDF[PrismDF["Month"] == 3].copy()
AprPRDF = PrismDF[PrismDF["Month"] == 4].copy()
MayPRDF = PrismDF[PrismDF["Month"] == 5].copy()
JunPRDF = PrismDF[PrismDF["Month"] == 6].copy()
JulPRDF = PrismDF[PrismDF["Month"] == 7].copy()
AugPRDF = PrismDF[PrismDF["Month"] == 8].copy()
SepPRDF = PrismDF[PrismDF["Month"] == 9].copy()
OctPRDF = PrismDF[PrismDF["Month"] == 10].copy()
NovPRDF = PrismDF[PrismDF["Month"] == 11].copy()
DecPRDF = PrismDF[PrismDF["Month"] == 12].copy()

In [ ]:
del PrismDF

In [ ]:
JANCmipDF = pd.read_pickle( JAN_CMIP5_IN )

In [ ]:
display( HTML( JANCmipDF.tail().to_html() ) )

In [ ]:
FEBCmipDF = pd.read_pickle( FEB_CMIP5_IN )
MARCmipDF = pd.read_pickle( MAR_CMIP5_IN )
APRCmipDF = pd.read_pickle( APR_CMIP5_IN )
MAYCmipDF = pd.read_pickle( MAY_CMIP5_IN )
JUNCmipDF = pd.read_pickle( JUN_CMIP5_IN )
JULCmipDF = pd.read_pickle( JUL_CMIP5_IN )
AUGCmipDF = pd.read_pickle( AUG_CMIP5_IN )
SEPCmipDF = pd.read_pickle( SEP_CMIP5_IN )
OCTCmipDF = pd.read_pickle( OCT_CMIP5_IN )
NOVCmipDF = pd.read_pickle( NOV_CMIP5_IN )
DECCmipDF = pd.read_pickle( DEC_CMIP5_IN )

Need to split CMIP5 data by grid id and then by month.

In [ ]:
JanLODF = JANCmipDF[JANCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
FebLODF = FEBCmipDF[FEBCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
MarLODF = MARCmipDF[MARCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
AprLODF = APRCmipDF[APRCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
MayLODF = MAYCmipDF[MAYCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
JunLODF = JUNCmipDF[JUNCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
JulLODF = JULCmipDF[JULCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
AugLODF = AUGCmipDF[AUGCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
SepLODF = SEPCmipDF[SEPCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
OctLODF = OCTCmipDF[OCTCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
NovLODF = NOVCmipDF[NOVCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()
DecLODF = DECCmipDF[DECCmipDF["Grid_Id"] <= LOCA_GRID_END].copy()

In [ ]:
JanBCDF = JANCmipDF[JANCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
FebBCDF = FEBCmipDF[FEBCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
MarBCDF = MARCmipDF[MARCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
AprBCDF = APRCmipDF[APRCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
MayBCDF = MAYCmipDF[MAYCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
JunBCDF = JUNCmipDF[JUNCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
JulBCDF = JULCmipDF[JULCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
AugBCDF = AUGCmipDF[AUGCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
SepBCDF = SEPCmipDF[SEPCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
OctBCDF = OCTCmipDF[OCTCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
NovBCDF = NOVCmipDF[NOVCmipDF["Grid_Id"] > LOCA_GRID_END].copy()
DecBCDF = DECCmipDF[DECCmipDF["Grid_Id"] > LOCA_GRID_END].copy()

In [ ]:
del JANCmipDF
del FEBCmipDF
del MARCmipDF
del APRCmipDF
del MAYCmipDF
del JUNCmipDF
del JULCmipDF
del AUGCmipDF
del SEPCmipDF
del OCTCmipDF
del NOVCmipDF
del DECCmipDF

## Comparisons by Month

In [ ]:
SPRMean = list()
SPRMeanMinCI = list()
SPRMeanMaxCI = list()
SPRVar = list()
SPRVarMinCI = list()
SPRVarMaxCI = list()
SPRStd = list()
SPRStdMinCI = list()
SPRStdMaxCI = list()

In [ ]:
SLOMean = list()
SLOMeanMinCI = list()
SLOMeanMaxCI = list()
SLOVar = list()
SLOVarMinCI = list()
SLOVarMaxCI = list()
SLOStd = list()
SLOStdMinCI = list()
SLOStdMaxCI = list()

In [ ]:
SBCMean = list()
SBCMeanMinCI = list()
SBCMeanMaxCI = list()
SBCVar = list()
SBCVarMinCI = list()
SBCVarMaxCI = list()
SBCStd = list()
SBCStdMinCI = list()
SBCStdMaxCI = list()

In [ ]:
CompQs = [ round(x * 0.05, 2) for x in range(21) ]
nCompQs = np.array( CompQs, dtype=np.float32 )
nCompQs

### Jan

Get some theoretical or fit parameters to use later for plotting

In [ ]:
JanNum = 1

In [ ]:
PRConvP = PRDistDF.at[JanNum , HDRS[10]]
PRConvR = PRDistDF.at[JanNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[JanNum , HDRS[10]]
LOConvR = LODistDF.at[JanNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[JanNum , HDRS[10]]
BCConvR = BCDistDF.at[JanNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
JanPrBSCI = sstats.bayes_mvs( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
JanPrBSCI

In [ ]:
JanPrBSCI[0][0], JanPrBSCI[0][1][0], JanPrBSCI[0][1][1]

In [ ]:
SPRMean.append( JanPrBSCI[0][0] )
SPRMeanMinCI.append( JanPrBSCI[0][1][0] )
SPRMeanMaxCI.append( JanPrBSCI[0][1][1] )
SPRVar.append( JanPrBSCI[1][0] )
SPRVarMinCI.append( JanPrBSCI[1][1][0] )
SPRVarMaxCI.append( JanPrBSCI[1][1][1] )
SPRStd.append( JanPrBSCI[2][0] )
SPRStdMinCI.append( JanPrBSCI[2][1][0] )
SPRStdMaxCI.append( JanPrBSCI[2][1][1] )

In [ ]:
JanLOBSCI = sstats.bayes_mvs( np.array( JanLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( JanLOBSCI[0][0] )
SLOMeanMinCI.append( JanLOBSCI[0][1][0] )
SLOMeanMaxCI.append( JanLOBSCI[0][1][1] )
SLOVar.append( JanLOBSCI[1][0] )
SLOVarMinCI.append( JanLOBSCI[1][1][0] )
SLOVarMaxCI.append( JanLOBSCI[1][1][1] )
SLOStd.append( JanLOBSCI[2][0] )
SLOStdMinCI.append( JanLOBSCI[2][1][0] )
SLOStdMaxCI.append( JanLOBSCI[2][1][1] )

In [ ]:
JanBCBSCI = sstats.bayes_mvs( np.array( JanBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( JanBCBSCI[0][0] )
SBCMeanMinCI.append( JanBCBSCI[0][1][0] )
SBCMeanMaxCI.append( JanBCBSCI[0][1][1] )
SBCVar.append( JanBCBSCI[1][0] )
SBCVarMinCI.append( JanBCBSCI[1][1][0] )
SBCVarMaxCI.append( JanBCBSCI[1][1][1] )
SBCStd.append( JanBCBSCI[2][0] )
SBCStdMinCI.append( JanBCBSCI[2][1][0] )
SBCStdMaxCI.append( JanBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
JanTTPRtoLO = sstats.ttest_ind( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JanLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JanTTPRtoLOPval = JanTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % JanTTPRtoLOPval )

In [ ]:
JanTTPRtoBC = sstats.ttest_ind( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JanBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JanTTPRtoBCPval = JanTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % JanTTPRtoBCPval )

In [ ]:
JanTTLOtoBC = sstats.ttest_ind( np.array( JanLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JanBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JanTTLOtoBCPval = JanTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % JanTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
JanMWUPRtoLO = sstats.mannwhitneyu( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JanLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JanMWUPRtoLOPval = JanMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % JanMWUPRtoLOPval )

In [ ]:
JanMWUPRtoBC = sstats.mannwhitneyu( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JanBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JanMWUPRtoBCPval = JanMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % JanMWUPRtoBCPval )

In [ ]:
JanMWULOtoBC = sstats.mannwhitneyu( np.array( JanLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JanBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JanMWULOtoBCPval = JanMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % JanMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
JanKWPRtoLO = sstats.kruskal( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JanLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JanKWPRtoLOPval = JanKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % JanKWPRtoLOPval )

In [ ]:
JanKWPRtoBC = sstats.kruskal( np.array( JanPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JanBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JanKWPRtoBCPval = JanKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % JanKWPRtoBCPval )

In [ ]:
JanKWLOtoBC = sstats.kruskal( np.array( JanLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JanBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JanKWLOtoBCPval = JanKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % JanKWLOtoBCPval )

In [ ]:
JanLODF["Wet_Count"].median(), JanBCDF["Wet_Count"].median(), JanPRDF["Wet_Count"].median()

#### Plots

In [ ]:
JanPRQs = JanPRDF["Wet_Count"].quantile(q=nCompQs)
JanLOQs = JanLODF["Wet_Count"].quantile(q=nCompQs)
JanBCQs = JanBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( JanPRQs.max(), JanLOQs.max(), JanBCQs.max() )
maxQs = maxQs + 4.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( JanPRQs, JanLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( JanPRQs, JanBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( JanLOQs, JanBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Jan Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( JanPRQs, JanLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( JanPRQs, JanBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( JanLOQs, JanBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Jan Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 8.0) )
ax11.set_ylim( (0.0, 8.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( JanPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( JanLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( JanBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Jan Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Jan Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
width = 0.50
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS, PRpmf, width, color="xkcd:royal blue", label="PRISM" )
ax11.set_title('Jan Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Feb

Get some theoretical or fit parameters to use later for plotting

In [ ]:
FebNum = 2

In [ ]:
PRConvP = PRDistDF.at[FebNum , HDRS[10]]
PRConvR = PRDistDF.at[FebNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[FebNum , HDRS[10]]
LOConvR = LODistDF.at[FebNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[FebNum , HDRS[10]]
BCConvR = BCDistDF.at[FebNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
FebPrBSCI = sstats.bayes_mvs( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
FebPrBSCI

In [ ]:
FebPrBSCI[0][0], FebPrBSCI[0][1][0], FebPrBSCI[0][1][1]

In [ ]:
SPRMean.append( FebPrBSCI[0][0] )
SPRMeanMinCI.append( FebPrBSCI[0][1][0] )
SPRMeanMaxCI.append( FebPrBSCI[0][1][1] )
SPRVar.append( FebPrBSCI[1][0] )
SPRVarMinCI.append( FebPrBSCI[1][1][0] )
SPRVarMaxCI.append( FebPrBSCI[1][1][1] )
SPRStd.append( FebPrBSCI[2][0] )
SPRStdMinCI.append( FebPrBSCI[2][1][0] )
SPRStdMaxCI.append( FebPrBSCI[2][1][1] )

In [ ]:
FebLOBSCI = sstats.bayes_mvs( np.array( FebLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( FebLOBSCI[0][0] )
SLOMeanMinCI.append( FebLOBSCI[0][1][0] )
SLOMeanMaxCI.append( FebLOBSCI[0][1][1] )
SLOVar.append( FebLOBSCI[1][0] )
SLOVarMinCI.append( FebLOBSCI[1][1][0] )
SLOVarMaxCI.append( FebLOBSCI[1][1][1] )
SLOStd.append( FebLOBSCI[2][0] )
SLOStdMinCI.append( FebLOBSCI[2][1][0] )
SLOStdMaxCI.append( FebLOBSCI[2][1][1] )

In [ ]:
FebBCBSCI = sstats.bayes_mvs( np.array( FebBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( FebBCBSCI[0][0] )
SBCMeanMinCI.append( FebBCBSCI[0][1][0] )
SBCMeanMaxCI.append( FebBCBSCI[0][1][1] )
SBCVar.append( FebBCBSCI[1][0] )
SBCVarMinCI.append( FebBCBSCI[1][1][0] )
SBCVarMaxCI.append( FebBCBSCI[1][1][1] )
SBCStd.append( FebBCBSCI[2][0] )
SBCStdMinCI.append( FebBCBSCI[2][1][0] )
SBCStdMaxCI.append( FebBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
FebTTPRtoLO = sstats.ttest_ind( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( FebLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
FebTTPRtoLOPval = FebTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % FebTTPRtoLOPval )

In [ ]:
FebTTPRtoBC = sstats.ttest_ind( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( FebBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
FebTTPRtoBCPval = FebTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % FebTTPRtoBCPval )

In [ ]:
FebTTLOtoBC = sstats.ttest_ind( np.array( FebLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( FebBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
FebTTLOtoBCPval = FebTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % FebTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
FebMWUPRtoLO = sstats.mannwhitneyu( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( FebLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
FebMWUPRtoLOPval = FebMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % FebMWUPRtoLOPval )

In [ ]:
FebMWUPRtoBC = sstats.mannwhitneyu( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( FebBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
FebMWUPRtoBCPval = FebMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % FebMWUPRtoBCPval )

In [ ]:
FebMWULOtoBC = sstats.mannwhitneyu( np.array( FebLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( FebBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
FebMWULOtoBCPval = FebMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % FebMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
FebKWPRtoLO = sstats.kruskal( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( FebLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
FebKWPRtoLOPval = FebKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % FebKWPRtoLOPval )

In [ ]:
FebKWPRtoBC = sstats.kruskal( np.array( FebPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( FebBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
FebKWPRtoBCPval = FebKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % FebKWPRtoBCPval )

In [ ]:
FebKWLOtoBC = sstats.kruskal( np.array( FebLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( FebBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
FebKWLOtoBCPval = FebKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % FebKWLOtoBCPval )

In [ ]:
FebLODF["Wet_Count"].median(), FebBCDF["Wet_Count"].median(), FebPRDF["Wet_Count"].median()

#### Plots

In [ ]:
FebPRQs = FebPRDF["Wet_Count"].quantile(q=nCompQs)
FebLOQs = FebLODF["Wet_Count"].quantile(q=nCompQs)
FebBCQs = FebBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( FebPRQs.max(), FebLOQs.max(), FebBCQs.max() )
maxQs = maxQs + 5.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( FebPRQs, FebLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( FebPRQs, FebBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( FebLOQs, FebBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Feb Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( FebPRQs, FebLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( FebPRQs, FebBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( FebLOQs, FebBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Feb Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 8.0) )
ax11.set_ylim( (0.0, 8.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( FebPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( FebLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( FebBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Feb Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Feb Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Mar

Get some theoretical or fit parameters to use later for plotting

In [ ]:
MarNum = 3

In [ ]:
PRConvP = PRDistDF.at[MarNum , HDRS[10]]
PRConvR = PRDistDF.at[MarNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[MarNum , HDRS[10]]
LOConvR = LODistDF.at[MarNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[MarNum , HDRS[10]]
BCConvR = BCDistDF.at[MarNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
MarPrBSCI = sstats.bayes_mvs( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
MarPrBSCI

In [ ]:
MarPrBSCI[0][0], MarPrBSCI[0][1][0], MarPrBSCI[0][1][1]

In [ ]:
SPRMean.append( MarPrBSCI[0][0] )
SPRMeanMinCI.append( MarPrBSCI[0][1][0] )
SPRMeanMaxCI.append( MarPrBSCI[0][1][1] )
SPRVar.append( MarPrBSCI[1][0] )
SPRVarMinCI.append( MarPrBSCI[1][1][0] )
SPRVarMaxCI.append( MarPrBSCI[1][1][1] )
SPRStd.append( MarPrBSCI[2][0] )
SPRStdMinCI.append( MarPrBSCI[2][1][0] )
SPRStdMaxCI.append( MarPrBSCI[2][1][1] )

In [ ]:
MarLOBSCI = sstats.bayes_mvs( np.array( MarLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( MarLOBSCI[0][0] )
SLOMeanMinCI.append( MarLOBSCI[0][1][0] )
SLOMeanMaxCI.append( MarLOBSCI[0][1][1] )
SLOVar.append( MarLOBSCI[1][0] )
SLOVarMinCI.append( MarLOBSCI[1][1][0] )
SLOVarMaxCI.append( MarLOBSCI[1][1][1] )
SLOStd.append( MarLOBSCI[2][0] )
SLOStdMinCI.append( MarLOBSCI[2][1][0] )
SLOStdMaxCI.append( MarLOBSCI[2][1][1] )

In [ ]:
MarBCBSCI = sstats.bayes_mvs( np.array( MarBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( MarBCBSCI[0][0] )
SBCMeanMinCI.append( MarBCBSCI[0][1][0] )
SBCMeanMaxCI.append( MarBCBSCI[0][1][1] )
SBCVar.append( MarBCBSCI[1][0] )
SBCVarMinCI.append( MarBCBSCI[1][1][0] )
SBCVarMaxCI.append( MarBCBSCI[1][1][1] )
SBCStd.append( MarBCBSCI[2][0] )
SBCStdMinCI.append( MarBCBSCI[2][1][0] )
SBCStdMaxCI.append( MarBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
MarTTPRtoLO = sstats.ttest_ind( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( MarLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MarTTPRtoLOPval = MarTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % MarTTPRtoLOPval )

In [ ]:
MarTTPRtoBC = sstats.ttest_ind( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( MarBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MarTTPRtoBCPval = MarTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % MarTTPRtoBCPval )

In [ ]:
MarTTLOtoBC = sstats.ttest_ind( np.array( MarLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( MarBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MarTTLOtoBCPval = MarTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % MarTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
MarMWUPRtoLO = sstats.mannwhitneyu( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( MarLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
MarMWUPRtoLOPval = MarMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % MarMWUPRtoLOPval )

In [ ]:
MarMWUPRtoBC = sstats.mannwhitneyu( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( MarBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
MarMWUPRtoBCPval = MarMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % MarMWUPRtoBCPval )

In [ ]:
MarMWULOtoBC = sstats.mannwhitneyu( np.array( MarLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( MarBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
MarMWULOtoBCPval = MarMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % MarMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
MarKWPRtoLO = sstats.kruskal( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( MarLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MarKWPRtoLOPval = MarKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % MarKWPRtoLOPval )

In [ ]:
MarKWPRtoBC = sstats.kruskal( np.array( MarPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( MarBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MarKWPRtoBCPval = MarKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % MarKWPRtoBCPval )

In [ ]:
MarKWLOtoBC = sstats.kruskal( np.array( MarLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( MarBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MarKWLOtoBCPval = MarKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % MarKWLOtoBCPval )

In [ ]:
MarLODF["Wet_Count"].median(), MarBCDF["Wet_Count"].median(), MarPRDF["Wet_Count"].median()

#### Plots

In [ ]:
MarPRQs = MarPRDF["Wet_Count"].quantile(q=nCompQs)
MarLOQs = MarLODF["Wet_Count"].quantile(q=nCompQs)
MarBCQs = MarBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( MarPRQs.max(), MarLOQs.max(), MarBCQs.max() )
maxQs = maxQs + 5.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( MarPRQs, MarLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( MarPRQs, MarBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( MarLOQs, MarBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Mar Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( MarPRQs, MarLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( MarPRQs, MarBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( MarLOQs, MarBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Mar Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( MarPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( MarLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( MarBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Mar Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Mar Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Apr

Get some theoretical or fit parameters to use later for plotting

In [ ]:
AprNum = 4

In [ ]:
PRConvP = PRDistDF.at[AprNum , HDRS[10]]
PRConvR = PRDistDF.at[AprNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[AprNum , HDRS[10]]
LOConvR = LODistDF.at[AprNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[AprNum , HDRS[10]]
BCConvR = BCDistDF.at[AprNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
AprPrBSCI = sstats.bayes_mvs( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
AprPrBSCI

In [ ]:
AprPrBSCI[0][0], AprPrBSCI[0][1][0], AprPrBSCI[0][1][1]

In [ ]:
SPRMean.append( AprPrBSCI[0][0] )
SPRMeanMinCI.append( AprPrBSCI[0][1][0] )
SPRMeanMaxCI.append( AprPrBSCI[0][1][1] )
SPRVar.append( AprPrBSCI[1][0] )
SPRVarMinCI.append( AprPrBSCI[1][1][0] )
SPRVarMaxCI.append( AprPrBSCI[1][1][1] )
SPRStd.append( AprPrBSCI[2][0] )
SPRStdMinCI.append( AprPrBSCI[2][1][0] )
SPRStdMaxCI.append( AprPrBSCI[2][1][1] )

In [ ]:
AprLOBSCI = sstats.bayes_mvs( np.array( AprLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( AprLOBSCI[0][0] )
SLOMeanMinCI.append( AprLOBSCI[0][1][0] )
SLOMeanMaxCI.append( AprLOBSCI[0][1][1] )
SLOVar.append( AprLOBSCI[1][0] )
SLOVarMinCI.append( AprLOBSCI[1][1][0] )
SLOVarMaxCI.append( AprLOBSCI[1][1][1] )
SLOStd.append( AprLOBSCI[2][0] )
SLOStdMinCI.append( AprLOBSCI[2][1][0] )
SLOStdMaxCI.append( AprLOBSCI[2][1][1] )

In [ ]:
AprBCBSCI = sstats.bayes_mvs( np.array( AprBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( AprBCBSCI[0][0] )
SBCMeanMinCI.append( AprBCBSCI[0][1][0] )
SBCMeanMaxCI.append( AprBCBSCI[0][1][1] )
SBCVar.append( AprBCBSCI[1][0] )
SBCVarMinCI.append( AprBCBSCI[1][1][0] )
SBCVarMaxCI.append( AprBCBSCI[1][1][1] )
SBCStd.append( AprBCBSCI[2][0] )
SBCStdMinCI.append( AprBCBSCI[2][1][0] )
SBCStdMaxCI.append( AprBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
AprTTPRtoLO = sstats.ttest_ind( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( AprLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AprTTPRtoLOPval = AprTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % AprTTPRtoLOPval )

In [ ]:
AprTTPRtoBC = sstats.ttest_ind( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( AprBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AprTTPRtoBCPval = AprTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % AprTTPRtoBCPval )

In [ ]:
AprTTLOtoBC = sstats.ttest_ind( np.array( AprLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( AprBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AprTTLOtoBCPval = AprTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % AprTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
AprMWUPRtoLO = sstats.mannwhitneyu( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( AprLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
AprMWUPRtoLOPval = AprMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % AprMWUPRtoLOPval )

In [ ]:
AprMWUPRtoBC = sstats.mannwhitneyu( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( AprBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
AprMWUPRtoBCPval = AprMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % AprMWUPRtoBCPval )

In [ ]:
AprMWULOtoBC = sstats.mannwhitneyu( np.array( AprLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( AprBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
AprMWULOtoBCPval = AprMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % AprMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
AprKWPRtoLO = sstats.kruskal( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( AprLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AprKWPRtoLOPval = AprKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % AprKWPRtoLOPval )

In [ ]:
AprKWPRtoBC = sstats.kruskal( np.array( AprPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( AprBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AprKWPRtoBCPval = AprKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % AprKWPRtoBCPval )

In [ ]:
AprKWLOtoBC = sstats.kruskal( np.array( AprLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( AprBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AprKWLOtoBCPval = AprKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % AprKWLOtoBCPval )

In [ ]:
AprLODF["Wet_Count"].median(), AprBCDF["Wet_Count"].median(), AprPRDF["Wet_Count"].median()

#### Plots

In [ ]:
AprPRQs = AprPRDF["Wet_Count"].quantile(q=nCompQs)
AprLOQs = AprLODF["Wet_Count"].quantile(q=nCompQs)
AprBCQs = AprBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( AprPRQs.max(), AprLOQs.max(), AprBCQs.max() )
maxQs = maxQs + 3.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( AprPRQs, AprLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( AprPRQs, AprBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( AprLOQs, AprBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Apr Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( AprPRQs, AprLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( AprPRQs, AprBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( AprLOQs, AprBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Apr Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( AprPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( AprLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( AprBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Apr Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Apr Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### May

Get some theoretical or fit parameters to use later for plotting

In [ ]:
MayNum = 5

In [ ]:
PRConvP = PRDistDF.at[MayNum , HDRS[10]]
PRConvR = PRDistDF.at[MayNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[MayNum , HDRS[10]]
LOConvR = LODistDF.at[MayNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[MayNum , HDRS[10]]
BCConvR = BCDistDF.at[MayNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
MayPrBSCI = sstats.bayes_mvs( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
MayPrBSCI

In [ ]:
MayPrBSCI[0][0], MayPrBSCI[0][1][0], MayPrBSCI[0][1][1]

In [ ]:
SPRMean.append( MayPrBSCI[0][0] )
SPRMeanMinCI.append( MayPrBSCI[0][1][0] )
SPRMeanMaxCI.append( MayPrBSCI[0][1][1] )
SPRVar.append( MayPrBSCI[1][0] )
SPRVarMinCI.append( MayPrBSCI[1][1][0] )
SPRVarMaxCI.append( MayPrBSCI[1][1][1] )
SPRStd.append( MayPrBSCI[2][0] )
SPRStdMinCI.append( MayPrBSCI[2][1][0] )
SPRStdMaxCI.append( MayPrBSCI[2][1][1] )

In [ ]:
MayLOBSCI = sstats.bayes_mvs( np.array( MayLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( MayLOBSCI[0][0] )
SLOMeanMinCI.append( MayLOBSCI[0][1][0] )
SLOMeanMaxCI.append( MayLOBSCI[0][1][1] )
SLOVar.append( MayLOBSCI[1][0] )
SLOVarMinCI.append( MayLOBSCI[1][1][0] )
SLOVarMaxCI.append( MayLOBSCI[1][1][1] )
SLOStd.append( MayLOBSCI[2][0] )
SLOStdMinCI.append( MayLOBSCI[2][1][0] )
SLOStdMaxCI.append( MayLOBSCI[2][1][1] )

In [ ]:
MayBCBSCI = sstats.bayes_mvs( np.array( MayBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( MayBCBSCI[0][0] )
SBCMeanMinCI.append( MayBCBSCI[0][1][0] )
SBCMeanMaxCI.append( MayBCBSCI[0][1][1] )
SBCVar.append( MayBCBSCI[1][0] )
SBCVarMinCI.append( MayBCBSCI[1][1][0] )
SBCVarMaxCI.append( MayBCBSCI[1][1][1] )
SBCStd.append( MayBCBSCI[2][0] )
SBCStdMinCI.append( MayBCBSCI[2][1][0] )
SBCStdMaxCI.append( MayBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
MayTTPRtoLO = sstats.ttest_ind( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( MayLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MayTTPRtoLOPval = MayTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % MayTTPRtoLOPval )

In [ ]:
MayTTPRtoBC = sstats.ttest_ind( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( MayBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MayTTPRtoBCPval = MayTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % MayTTPRtoBCPval )

In [ ]:
MayTTLOtoBC = sstats.ttest_ind( np.array( MayLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( MayBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MayTTLOtoBCPval = MayTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % MayTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
MayMWUPRtoLO = sstats.mannwhitneyu( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( MayLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
MayMWUPRtoLOPval = MayMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % MayMWUPRtoLOPval )

In [ ]:
MayMWUPRtoBC = sstats.mannwhitneyu( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( MayBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
MayMWUPRtoBCPval = MayMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % MayMWUPRtoBCPval )

In [ ]:
MayMWULOtoBC = sstats.mannwhitneyu( np.array( MayLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( MayBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
MayMWULOtoBCPval = MayMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % MayMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
MayKWPRtoLO = sstats.kruskal( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( MayLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MayKWPRtoLOPval = MayKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % MayKWPRtoLOPval )

In [ ]:
MayKWPRtoBC = sstats.kruskal( np.array( MayPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( MayBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MayKWPRtoBCPval = MayKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % MayKWPRtoBCPval )

In [ ]:
MayKWLOtoBC = sstats.kruskal( np.array( MayLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( MayBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
MayKWLOtoBCPval = MayKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % MayKWLOtoBCPval )

In [ ]:
MayLODF["Wet_Count"].median(), MayBCDF["Wet_Count"].median(), MayPRDF["Wet_Count"].median()

#### Plots

In [ ]:
MayPRQs = MayPRDF["Wet_Count"].quantile(q=nCompQs)
MayLOQs = MayLODF["Wet_Count"].quantile(q=nCompQs)
MayBCQs = MayBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( MayPRQs.max(), MayLOQs.max(), MayBCQs.max() )
maxQs = maxQs + 4.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( MayPRQs, MayLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( MayPRQs, MayBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( MayLOQs, MayBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('May Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( MayPRQs, MayLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( MayPRQs, MayBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( MayLOQs, MayBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('May Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( MayPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( MayLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( MayBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('May Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('May Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Jun

Get some theoretical or fit parameters to use later for plotting

In [ ]:
JunNum = 6

In [ ]:
PRConvP = PRDistDF.at[JunNum , HDRS[10]]
PRConvR = PRDistDF.at[JunNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[JunNum , HDRS[10]]
LOConvR = LODistDF.at[JunNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[JunNum , HDRS[10]]
BCConvR = BCDistDF.at[JunNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
JunPrBSCI = sstats.bayes_mvs( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
JunPrBSCI

In [ ]:
JunPrBSCI[0][0], JunPrBSCI[0][1][0], JunPrBSCI[0][1][1]

In [ ]:
SPRMean.append( JunPrBSCI[0][0] )
SPRMeanMinCI.append( JunPrBSCI[0][1][0] )
SPRMeanMaxCI.append( JunPrBSCI[0][1][1] )
SPRVar.append( JunPrBSCI[1][0] )
SPRVarMinCI.append( JunPrBSCI[1][1][0] )
SPRVarMaxCI.append( JunPrBSCI[1][1][1] )
SPRStd.append( JunPrBSCI[2][0] )
SPRStdMinCI.append( JunPrBSCI[2][1][0] )
SPRStdMaxCI.append( JunPrBSCI[2][1][1] )

In [ ]:
JunLOBSCI = sstats.bayes_mvs( np.array( JunLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( JunLOBSCI[0][0] )
SLOMeanMinCI.append( JunLOBSCI[0][1][0] )
SLOMeanMaxCI.append( JunLOBSCI[0][1][1] )
SLOVar.append( JunLOBSCI[1][0] )
SLOVarMinCI.append( JunLOBSCI[1][1][0] )
SLOVarMaxCI.append( JunLOBSCI[1][1][1] )
SLOStd.append( JunLOBSCI[2][0] )
SLOStdMinCI.append( JunLOBSCI[2][1][0] )
SLOStdMaxCI.append( JunLOBSCI[2][1][1] )

In [ ]:
JunBCBSCI = sstats.bayes_mvs( np.array( JunBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( JunBCBSCI[0][0] )
SBCMeanMinCI.append( JunBCBSCI[0][1][0] )
SBCMeanMaxCI.append( JunBCBSCI[0][1][1] )
SBCVar.append( JunBCBSCI[1][0] )
SBCVarMinCI.append( JunBCBSCI[1][1][0] )
SBCVarMaxCI.append( JunBCBSCI[1][1][1] )
SBCStd.append( JunBCBSCI[2][0] )
SBCStdMinCI.append( JunBCBSCI[2][1][0] )
SBCStdMaxCI.append( JunBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
JunTTPRtoLO = sstats.ttest_ind( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JunLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JunTTPRtoLOPval = JunTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % JunTTPRtoLOPval )

In [ ]:
JunTTPRtoBC = sstats.ttest_ind( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JunBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JunTTPRtoBCPval = JunTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % JunTTPRtoBCPval )

In [ ]:
JunTTLOtoBC = sstats.ttest_ind( np.array( JunLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JunBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JunTTLOtoBCPval = JunTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % JunTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
JunMWUPRtoLO = sstats.mannwhitneyu( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JunLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JunMWUPRtoLOPval = JunMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % JunMWUPRtoLOPval )

In [ ]:
JunMWUPRtoBC = sstats.mannwhitneyu( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JunBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JunMWUPRtoBCPval = JunMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % JunMWUPRtoBCPval )

In [ ]:
JunMWULOtoBC = sstats.mannwhitneyu( np.array( JunLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JunBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JunMWULOtoBCPval = JunMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % JunMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
JunKWPRtoLO = sstats.kruskal( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JunLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JunKWPRtoLOPval = JunKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % JunKWPRtoLOPval )

In [ ]:
JunKWPRtoBC = sstats.kruskal( np.array( JunPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JunBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JunKWPRtoBCPval = JunKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % JunKWPRtoBCPval )

In [ ]:
JunKWLOtoBC = sstats.kruskal( np.array( JunLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JunBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JunKWLOtoBCPval = JunKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % JunKWLOtoBCPval )

In [ ]:
JunLODF["Wet_Count"].median(), JunBCDF["Wet_Count"].median(), JunPRDF["Wet_Count"].median()

#### Plots

In [ ]:
JunPRQs = JunPRDF["Wet_Count"].quantile(q=nCompQs)
JunLOQs = JunLODF["Wet_Count"].quantile(q=nCompQs)
JunBCQs = JunBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( JunPRQs.max(), JunLOQs.max(), JunBCQs.max() )
maxQs = maxQs + 1.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( JunPRQs, JunLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( JunPRQs, JunBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( JunLOQs, JunBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Jun Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( JunPRQs, JunLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( JunPRQs, JunBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( JunLOQs, JunBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Jun Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( JunPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( JunLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( JunBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Jun Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Jun Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Jul

Get some theoretical or fit parameters to use later for plotting

In [ ]:
JulNum = 7

In [ ]:
PRConvP = PRDistDF.at[JulNum , HDRS[10]]
PRConvR = PRDistDF.at[JulNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[JulNum , HDRS[10]]
LOConvR = LODistDF.at[JulNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[JulNum , HDRS[10]]
BCConvR = BCDistDF.at[JulNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
JulPrBSCI = sstats.bayes_mvs( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
JulPrBSCI

In [ ]:
JulPrBSCI[0][0], JulPrBSCI[0][1][0], JulPrBSCI[0][1][1]

In [ ]:
SPRMean.append( JulPrBSCI[0][0] )
SPRMeanMinCI.append( JulPrBSCI[0][1][0] )
SPRMeanMaxCI.append( JulPrBSCI[0][1][1] )
SPRVar.append( JulPrBSCI[1][0] )
SPRVarMinCI.append( JulPrBSCI[1][1][0] )
SPRVarMaxCI.append( JulPrBSCI[1][1][1] )
SPRStd.append( JulPrBSCI[2][0] )
SPRStdMinCI.append( JulPrBSCI[2][1][0] )
SPRStdMaxCI.append( JulPrBSCI[2][1][1] )

In [ ]:
JulLOBSCI = sstats.bayes_mvs( np.array( JulLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( JulLOBSCI[0][0] )
SLOMeanMinCI.append( JulLOBSCI[0][1][0] )
SLOMeanMaxCI.append( JulLOBSCI[0][1][1] )
SLOVar.append( JulLOBSCI[1][0] )
SLOVarMinCI.append( JulLOBSCI[1][1][0] )
SLOVarMaxCI.append( JulLOBSCI[1][1][1] )
SLOStd.append( JulLOBSCI[2][0] )
SLOStdMinCI.append( JulLOBSCI[2][1][0] )
SLOStdMaxCI.append( JulLOBSCI[2][1][1] )

In [ ]:
JulBCBSCI = sstats.bayes_mvs( np.array( JulBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( JulBCBSCI[0][0] )
SBCMeanMinCI.append( JulBCBSCI[0][1][0] )
SBCMeanMaxCI.append( JulBCBSCI[0][1][1] )
SBCVar.append( JulBCBSCI[1][0] )
SBCVarMinCI.append( JulBCBSCI[1][1][0] )
SBCVarMaxCI.append( JulBCBSCI[1][1][1] )
SBCStd.append( JulBCBSCI[2][0] )
SBCStdMinCI.append( JulBCBSCI[2][1][0] )
SBCStdMaxCI.append( JulBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
JulTTPRtoLO = sstats.ttest_ind( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JulLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JulTTPRtoLOPval = JulTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % JulTTPRtoLOPval )

In [ ]:
JulTTPRtoBC = sstats.ttest_ind( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JulBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JulTTPRtoBCPval = JulTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % JulTTPRtoBCPval )

In [ ]:
JulTTLOtoBC = sstats.ttest_ind( np.array( JulLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( JulBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JulTTLOtoBCPval = JulTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % JulTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
JulMWUPRtoLO = sstats.mannwhitneyu( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JulLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JulMWUPRtoLOPval = JulMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % JulMWUPRtoLOPval )

In [ ]:
JulMWUPRtoBC = sstats.mannwhitneyu( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JulBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JulMWUPRtoBCPval = JulMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % JulMWUPRtoBCPval )

In [ ]:
JulMWULOtoBC = sstats.mannwhitneyu( np.array( JulLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( JulBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
JulMWULOtoBCPval = JulMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % JulMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
JulKWPRtoLO = sstats.kruskal( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JulLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JulKWPRtoLOPval = JulKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % JulKWPRtoLOPval )

In [ ]:
JulKWPRtoBC = sstats.kruskal( np.array( JulPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JulBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JulKWPRtoBCPval = JulKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % JulKWPRtoBCPval )

In [ ]:
JulKWLOtoBC = sstats.kruskal( np.array( JulLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( JulBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
JulKWLOtoBCPval = JulKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % JulKWLOtoBCPval )

In [ ]:
JulLODF["Wet_Count"].median(), JulBCDF["Wet_Count"].median(), JulPRDF["Wet_Count"].median()

#### Plots

In [ ]:
JulPRQs = JulPRDF["Wet_Count"].quantile(q=nCompQs)
JulLOQs = JulLODF["Wet_Count"].quantile(q=nCompQs)
JulBCQs = JulBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( JulPRQs.max(), JulLOQs.max(), JulBCQs.max() )
maxQs = maxQs + 1.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( JulPRQs, JulLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( JulPRQs, JulBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( JulLOQs, JulBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Jul Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( JulPRQs, JulLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( JulPRQs, JulBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( JulLOQs, JulBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Jul Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( JulPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( JulLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( JulBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Jul Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Jul Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Aug

Get some theoretical or fit parameters to use later for plotting

In [ ]:
AugNum = 8

In [ ]:
PRConvP = PRDistDF.at[AugNum , HDRS[10]]
PRConvR = PRDistDF.at[AugNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[AugNum , HDRS[10]]
LOConvR = LODistDF.at[AugNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[AugNum , HDRS[10]]
BCConvR = BCDistDF.at[AugNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
AugPrBSCI = sstats.bayes_mvs( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
AugPrBSCI

In [ ]:
AugPrBSCI[0][0], AugPrBSCI[0][1][0], AugPrBSCI[0][1][1]

In [ ]:
SPRMean.append( AugPrBSCI[0][0] )
SPRMeanMinCI.append( AugPrBSCI[0][1][0] )
SPRMeanMaxCI.append( AugPrBSCI[0][1][1] )
SPRVar.append( AugPrBSCI[1][0] )
SPRVarMinCI.append( AugPrBSCI[1][1][0] )
SPRVarMaxCI.append( AugPrBSCI[1][1][1] )
SPRStd.append( AugPrBSCI[2][0] )
SPRStdMinCI.append( AugPrBSCI[2][1][0] )
SPRStdMaxCI.append( AugPrBSCI[2][1][1] )

In [ ]:
AugLOBSCI = sstats.bayes_mvs( np.array( AugLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( AugLOBSCI[0][0] )
SLOMeanMinCI.append( AugLOBSCI[0][1][0] )
SLOMeanMaxCI.append( AugLOBSCI[0][1][1] )
SLOVar.append( AugLOBSCI[1][0] )
SLOVarMinCI.append( AugLOBSCI[1][1][0] )
SLOVarMaxCI.append( AugLOBSCI[1][1][1] )
SLOStd.append( AugLOBSCI[2][0] )
SLOStdMinCI.append( AugLOBSCI[2][1][0] )
SLOStdMaxCI.append( AugLOBSCI[2][1][1] )

In [ ]:
AugBCBSCI = sstats.bayes_mvs( np.array( AugBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( AugBCBSCI[0][0] )
SBCMeanMinCI.append( AugBCBSCI[0][1][0] )
SBCMeanMaxCI.append( AugBCBSCI[0][1][1] )
SBCVar.append( AugBCBSCI[1][0] )
SBCVarMinCI.append( AugBCBSCI[1][1][0] )
SBCVarMaxCI.append( AugBCBSCI[1][1][1] )
SBCStd.append( AugBCBSCI[2][0] )
SBCStdMinCI.append( AugBCBSCI[2][1][0] )
SBCStdMaxCI.append( AugBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
AugTTPRtoLO = sstats.ttest_ind( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( AugLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AugTTPRtoLOPval = AugTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % AugTTPRtoLOPval )

In [ ]:
AugTTPRtoBC = sstats.ttest_ind( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( AugBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AugTTPRtoBCPval = AugTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % AugTTPRtoBCPval )

In [ ]:
AugTTLOtoBC = sstats.ttest_ind( np.array( AugLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( AugBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AugTTLOtoBCPval = AugTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % AugTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
AugMWUPRtoLO = sstats.mannwhitneyu( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( AugLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
AugMWUPRtoLOPval = AugMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % AugMWUPRtoLOPval )

In [ ]:
AugMWUPRtoBC = sstats.mannwhitneyu( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( AugBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
AugMWUPRtoBCPval = AugMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % AugMWUPRtoBCPval )

In [ ]:
AugMWULOtoBC = sstats.mannwhitneyu( np.array( AugLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( AugBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
AugMWULOtoBCPval = AugMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % AugMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
AugKWPRtoLO = sstats.kruskal( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( AugLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AugKWPRtoLOPval = AugKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % AugKWPRtoLOPval )

In [ ]:
AugKWPRtoBC = sstats.kruskal( np.array( AugPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( AugBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AugKWPRtoBCPval = AugKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % AugKWPRtoBCPval )

In [ ]:
AugKWLOtoBC = sstats.kruskal( np.array( AugLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( AugBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
AugKWLOtoBCPval = AugKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % AugKWLOtoBCPval )

In [ ]:
AugLODF["Wet_Count"].median(), AugBCDF["Wet_Count"].median(), AugPRDF["Wet_Count"].median()

#### Plots

In [ ]:
AugPRQs = AugPRDF["Wet_Count"].quantile(q=nCompQs)
AugLOQs = AugLODF["Wet_Count"].quantile(q=nCompQs)
AugBCQs = AugBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( AugPRQs.max(), AugLOQs.max(), AugBCQs.max() )
maxQs = maxQs + 1.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( AugPRQs, AugLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( AugPRQs, AugBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( AugLOQs, AugBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Aug Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( AugPRQs, AugLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( AugPRQs, AugBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( AugLOQs, AugBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Aug Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( AugPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( AugLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( AugBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Aug Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Aug Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Sep

Get some theoretical or fit parameters to use later for plotting

In [ ]:
SepNum = 9

In [ ]:
PRConvP = PRDistDF.at[SepNum , HDRS[10]]
PRConvR = PRDistDF.at[SepNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[SepNum , HDRS[10]]
LOConvR = LODistDF.at[SepNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[SepNum , HDRS[10]]
BCConvR = BCDistDF.at[SepNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
SepPrBSCI = sstats.bayes_mvs( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SepPrBSCI

In [ ]:
SepPrBSCI[0][0], SepPrBSCI[0][1][0], SepPrBSCI[0][1][1]

In [ ]:
SPRMean.append( SepPrBSCI[0][0] )
SPRMeanMinCI.append( SepPrBSCI[0][1][0] )
SPRMeanMaxCI.append( SepPrBSCI[0][1][1] )
SPRVar.append( SepPrBSCI[1][0] )
SPRVarMinCI.append( SepPrBSCI[1][1][0] )
SPRVarMaxCI.append( SepPrBSCI[1][1][1] )
SPRStd.append( SepPrBSCI[2][0] )
SPRStdMinCI.append( SepPrBSCI[2][1][0] )
SPRStdMaxCI.append( SepPrBSCI[2][1][1] )

In [ ]:
SepLOBSCI = sstats.bayes_mvs( np.array( SepLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( SepLOBSCI[0][0] )
SLOMeanMinCI.append( SepLOBSCI[0][1][0] )
SLOMeanMaxCI.append( SepLOBSCI[0][1][1] )
SLOVar.append( SepLOBSCI[1][0] )
SLOVarMinCI.append( SepLOBSCI[1][1][0] )
SLOVarMaxCI.append( SepLOBSCI[1][1][1] )
SLOStd.append( SepLOBSCI[2][0] )
SLOStdMinCI.append( SepLOBSCI[2][1][0] )
SLOStdMaxCI.append( SepLOBSCI[2][1][1] )

In [ ]:
SepBCBSCI = sstats.bayes_mvs( np.array( SepBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( SepBCBSCI[0][0] )
SBCMeanMinCI.append( SepBCBSCI[0][1][0] )
SBCMeanMaxCI.append( SepBCBSCI[0][1][1] )
SBCVar.append( SepBCBSCI[1][0] )
SBCVarMinCI.append( SepBCBSCI[1][1][0] )
SBCVarMaxCI.append( SepBCBSCI[1][1][1] )
SBCStd.append( SepBCBSCI[2][0] )
SBCStdMinCI.append( SepBCBSCI[2][1][0] )
SBCStdMaxCI.append( SepBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
SepTTPRtoLO = sstats.ttest_ind( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( SepLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
SepTTPRtoLOPval = SepTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % SepTTPRtoLOPval )

In [ ]:
SepTTPRtoBC = sstats.ttest_ind( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( SepBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
SepTTPRtoBCPval = SepTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % SepTTPRtoBCPval )

In [ ]:
SepTTLOtoBC = sstats.ttest_ind( np.array( SepLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( SepBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
SepTTLOtoBCPval = SepTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % SepTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
SepMWUPRtoLO = sstats.mannwhitneyu( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( SepLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
SepMWUPRtoLOPval = SepMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % SepMWUPRtoLOPval )

In [ ]:
SepMWUPRtoBC = sstats.mannwhitneyu( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( SepBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
SepMWUPRtoBCPval = SepMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % SepMWUPRtoBCPval )

In [ ]:
SepMWULOtoBC = sstats.mannwhitneyu( np.array( SepLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( SepBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
SepMWULOtoBCPval = SepMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % SepMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
SepKWPRtoLO = sstats.kruskal( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( SepLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
SepKWPRtoLOPval = SepKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % SepKWPRtoLOPval )

In [ ]:
SepKWPRtoBC = sstats.kruskal( np.array( SepPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( SepBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
SepKWPRtoBCPval = SepKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % SepKWPRtoBCPval )

In [ ]:
SepKWLOtoBC = sstats.kruskal( np.array( SepLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( SepBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
SepKWLOtoBCPval = SepKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % SepKWLOtoBCPval )

In [ ]:
SepLODF["Wet_Count"].median(), SepBCDF["Wet_Count"].median(), SepPRDF["Wet_Count"].median()

#### Plots

In [ ]:
SepPRQs = SepPRDF["Wet_Count"].quantile(q=nCompQs)
SepLOQs = SepLODF["Wet_Count"].quantile(q=nCompQs)
SepBCQs = SepBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( SepPRQs.max(), SepLOQs.max(), SepBCQs.max() )
maxQs = maxQs + 3.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( SepPRQs, SepLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( SepPRQs, SepBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( SepLOQs, SepBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Sep Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( SepPRQs, SepLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( SepPRQs, SepBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( SepLOQs, SepBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Sep Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( SepPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( SepLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( SepBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Sep Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Sep Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Oct

Get some theoretical or fit parameters to use later for plotting

In [ ]:
OctNum = 10

In [ ]:
PRConvP = PRDistDF.at[OctNum , HDRS[10]]
PRConvR = PRDistDF.at[OctNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[OctNum , HDRS[10]]
LOConvR = LODistDF.at[OctNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[OctNum , HDRS[10]]
BCConvR = BCDistDF.at[OctNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
OctPrBSCI = sstats.bayes_mvs( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
OctPrBSCI

In [ ]:
OctPrBSCI[0][0], OctPrBSCI[0][1][0], OctPrBSCI[0][1][1]

In [ ]:
SPRMean.append( OctPrBSCI[0][0] )
SPRMeanMinCI.append( OctPrBSCI[0][1][0] )
SPRMeanMaxCI.append( OctPrBSCI[0][1][1] )
SPRVar.append( OctPrBSCI[1][0] )
SPRVarMinCI.append( OctPrBSCI[1][1][0] )
SPRVarMaxCI.append( OctPrBSCI[1][1][1] )
SPRStd.append( OctPrBSCI[2][0] )
SPRStdMinCI.append( OctPrBSCI[2][1][0] )
SPRStdMaxCI.append( OctPrBSCI[2][1][1] )

In [ ]:
OctLOBSCI = sstats.bayes_mvs( np.array( OctLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( OctLOBSCI[0][0] )
SLOMeanMinCI.append( OctLOBSCI[0][1][0] )
SLOMeanMaxCI.append( OctLOBSCI[0][1][1] )
SLOVar.append( OctLOBSCI[1][0] )
SLOVarMinCI.append( OctLOBSCI[1][1][0] )
SLOVarMaxCI.append( OctLOBSCI[1][1][1] )
SLOStd.append( OctLOBSCI[2][0] )
SLOStdMinCI.append( OctLOBSCI[2][1][0] )
SLOStdMaxCI.append( OctLOBSCI[2][1][1] )

In [ ]:
OctBCBSCI = sstats.bayes_mvs( np.array( OctBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( OctBCBSCI[0][0] )
SBCMeanMinCI.append( OctBCBSCI[0][1][0] )
SBCMeanMaxCI.append( OctBCBSCI[0][1][1] )
SBCVar.append( OctBCBSCI[1][0] )
SBCVarMinCI.append( OctBCBSCI[1][1][0] )
SBCVarMaxCI.append( OctBCBSCI[1][1][1] )
SBCStd.append( OctBCBSCI[2][0] )
SBCStdMinCI.append( OctBCBSCI[2][1][0] )
SBCStdMaxCI.append( OctBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
OctTTPRtoLO = sstats.ttest_ind( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( OctLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
OctTTPRtoLOPval = OctTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % OctTTPRtoLOPval )

In [ ]:
OctTTPRtoBC = sstats.ttest_ind( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( OctBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
OctTTPRtoBCPval = OctTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % OctTTPRtoBCPval )

In [ ]:
OctTTLOtoBC = sstats.ttest_ind( np.array( OctLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( OctBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
OctTTLOtoBCPval = OctTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % OctTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
OctMWUPRtoLO = sstats.mannwhitneyu( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( OctLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
OctMWUPRtoLOPval = OctMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % OctMWUPRtoLOPval )

In [ ]:
OctMWUPRtoBC = sstats.mannwhitneyu( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( OctBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
OctMWUPRtoBCPval = OctMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % OctMWUPRtoBCPval )

In [ ]:
OctMWULOtoBC = sstats.mannwhitneyu( np.array( OctLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( OctBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
OctMWULOtoBCPval = OctMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % OctMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
OctKWPRtoLO = sstats.kruskal( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( OctLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
OctKWPRtoLOPval = OctKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % OctKWPRtoLOPval )

In [ ]:
OctKWPRtoBC = sstats.kruskal( np.array( OctPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( OctBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
OctKWPRtoBCPval = OctKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % OctKWPRtoBCPval )

In [ ]:
OctKWLOtoBC = sstats.kruskal( np.array( OctLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( OctBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
OctKWLOtoBCPval = OctKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % OctKWLOtoBCPval )

In [ ]:
OctLODF["Wet_Count"].median(), OctBCDF["Wet_Count"].median(), OctPRDF["Wet_Count"].median()

#### Plots

In [ ]:
OctPRQs = OctPRDF["Wet_Count"].quantile(q=nCompQs)
OctLOQs = OctLODF["Wet_Count"].quantile(q=nCompQs)
OctBCQs = OctBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( OctPRQs.max(), OctLOQs.max(), OctBCQs.max() )
maxQs = maxQs + 3.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( OctPRQs, OctLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( OctPRQs, OctBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( OctLOQs, OctBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Oct Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( OctPRQs, OctLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( OctPRQs, OctBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( OctLOQs, OctBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Oct Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( OctPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( OctLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( OctBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Oct Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Oct Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Nov

Get some theoretical or fit parameters to use later for plotting

In [ ]:
NovNum = 11

In [ ]:
PRConvP = PRDistDF.at[NovNum , HDRS[10]]
PRConvR = PRDistDF.at[NovNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[NovNum , HDRS[10]]
LOConvR = LODistDF.at[NovNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[NovNum , HDRS[10]]
BCConvR = BCDistDF.at[NovNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
NovPrBSCI = sstats.bayes_mvs( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
NovPrBSCI

In [ ]:
NovPrBSCI[0][0], NovPrBSCI[0][1][0], NovPrBSCI[0][1][1]

In [ ]:
SPRMean.append( NovPrBSCI[0][0] )
SPRMeanMinCI.append( NovPrBSCI[0][1][0] )
SPRMeanMaxCI.append( NovPrBSCI[0][1][1] )
SPRVar.append( NovPrBSCI[1][0] )
SPRVarMinCI.append( NovPrBSCI[1][1][0] )
SPRVarMaxCI.append( NovPrBSCI[1][1][1] )
SPRStd.append( NovPrBSCI[2][0] )
SPRStdMinCI.append( NovPrBSCI[2][1][0] )
SPRStdMaxCI.append( NovPrBSCI[2][1][1] )

In [ ]:
NovLOBSCI = sstats.bayes_mvs( np.array( NovLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( NovLOBSCI[0][0] )
SLOMeanMinCI.append( NovLOBSCI[0][1][0] )
SLOMeanMaxCI.append( NovLOBSCI[0][1][1] )
SLOVar.append( NovLOBSCI[1][0] )
SLOVarMinCI.append( NovLOBSCI[1][1][0] )
SLOVarMaxCI.append( NovLOBSCI[1][1][1] )
SLOStd.append( NovLOBSCI[2][0] )
SLOStdMinCI.append( NovLOBSCI[2][1][0] )
SLOStdMaxCI.append( NovLOBSCI[2][1][1] )

In [ ]:
NovBCBSCI = sstats.bayes_mvs( np.array( NovBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( NovBCBSCI[0][0] )
SBCMeanMinCI.append( NovBCBSCI[0][1][0] )
SBCMeanMaxCI.append( NovBCBSCI[0][1][1] )
SBCVar.append( NovBCBSCI[1][0] )
SBCVarMinCI.append( NovBCBSCI[1][1][0] )
SBCVarMaxCI.append( NovBCBSCI[1][1][1] )
SBCStd.append( NovBCBSCI[2][0] )
SBCStdMinCI.append( NovBCBSCI[2][1][0] )
SBCStdMaxCI.append( NovBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
NovTTPRtoLO = sstats.ttest_ind( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( NovLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
NovTTPRtoLOPval = NovTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % NovTTPRtoLOPval )

In [ ]:
NovTTPRtoBC = sstats.ttest_ind( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( NovBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
NovTTPRtoBCPval = NovTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % NovTTPRtoBCPval )

In [ ]:
NovTTLOtoBC = sstats.ttest_ind( np.array( NovLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( NovBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
NovTTLOtoBCPval = NovTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % NovTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
NovMWUPRtoLO = sstats.mannwhitneyu( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( NovLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
NovMWUPRtoLOPval = NovMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % NovMWUPRtoLOPval )

In [ ]:
NovMWUPRtoBC = sstats.mannwhitneyu( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( NovBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
NovMWUPRtoBCPval = NovMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % NovMWUPRtoBCPval )

In [ ]:
NovMWULOtoBC = sstats.mannwhitneyu( np.array( NovLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( NovBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
NovMWULOtoBCPval = NovMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % NovMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
NovKWPRtoLO = sstats.kruskal( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( NovLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
NovKWPRtoLOPval = NovKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % NovKWPRtoLOPval )

In [ ]:
NovKWPRtoBC = sstats.kruskal( np.array( NovPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( NovBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
NovKWPRtoBCPval = NovKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % NovKWPRtoBCPval )

In [ ]:
NovKWLOtoBC = sstats.kruskal( np.array( NovLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( NovBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
NovKWLOtoBCPval = NovKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % NovKWLOtoBCPval )

In [ ]:
NovLODF["Wet_Count"].median(), NovBCDF["Wet_Count"].median(), NovPRDF["Wet_Count"].median()

#### Plots

In [ ]:
NovPRQs = NovPRDF["Wet_Count"].quantile(q=nCompQs)
NovLOQs = NovLODF["Wet_Count"].quantile(q=nCompQs)
NovBCQs = NovBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( NovPRQs.max(), NovLOQs.max(), NovBCQs.max() )
maxQs = maxQs + 2.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( NovPRQs, NovLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( NovPRQs, NovBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( NovLOQs, NovBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Nov Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( NovPRQs, NovLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( NovPRQs, NovBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( NovLOQs, NovBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Nov Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( NovPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( NovLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( NovBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Nov Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Nov Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

### Dec

Get some theoretical or fit parameters to use later for plotting

In [ ]:
DecNum = 12

In [ ]:
PRConvP = PRDistDF.at[DecNum , HDRS[10]]
PRConvR = PRDistDF.at[DecNum , HDRS[11]]

In [ ]:
LOConvP = LODistDF.at[DecNum , HDRS[10]]
LOConvR = LODistDF.at[DecNum , HDRS[11]]

In [ ]:
BCConvP = BCDistDF.at[DecNum , HDRS[10]]
BCConvR = BCDistDF.at[DecNum , HDRS[11]]

Bayesian estimates of confidence intervals for mean, variance, and standard deviation of the samples

In [ ]:
DecPrBSCI = sstats.bayes_mvs( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
DecPrBSCI

In [ ]:
DecPrBSCI[0][0], DecPrBSCI[0][1][0], DecPrBSCI[0][1][1]

In [ ]:
SPRMean.append( DecPrBSCI[0][0] )
SPRMeanMinCI.append( DecPrBSCI[0][1][0] )
SPRMeanMaxCI.append( DecPrBSCI[0][1][1] )
SPRVar.append( DecPrBSCI[1][0] )
SPRVarMinCI.append( DecPrBSCI[1][1][0] )
SPRVarMaxCI.append( DecPrBSCI[1][1][1] )
SPRStd.append( DecPrBSCI[2][0] )
SPRStdMinCI.append( DecPrBSCI[2][1][0] )
SPRStdMaxCI.append( DecPrBSCI[2][1][1] )

In [ ]:
DecLOBSCI = sstats.bayes_mvs( np.array( DecLODF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SLOMean.append( DecLOBSCI[0][0] )
SLOMeanMinCI.append( DecLOBSCI[0][1][0] )
SLOMeanMaxCI.append( DecLOBSCI[0][1][1] )
SLOVar.append( DecLOBSCI[1][0] )
SLOVarMinCI.append( DecLOBSCI[1][1][0] )
SLOVarMaxCI.append( DecLOBSCI[1][1][1] )
SLOStd.append( DecLOBSCI[2][0] )
SLOStdMinCI.append( DecLOBSCI[2][1][0] )
SLOStdMaxCI.append( DecLOBSCI[2][1][1] )

In [ ]:
DecBCBSCI = sstats.bayes_mvs( np.array( DecBCDF["Wet_Count"], dtype=np.int32 ), alpha=0.9 )

In [ ]:
SBCMean.append( DecBCBSCI[0][0] )
SBCMeanMinCI.append( DecBCBSCI[0][1][0] )
SBCMeanMaxCI.append( DecBCBSCI[0][1][1] )
SBCVar.append( DecBCBSCI[1][0] )
SBCVarMinCI.append( DecBCBSCI[1][1][0] )
SBCVarMaxCI.append( DecBCBSCI[1][1][1] )
SBCStd.append( DecBCBSCI[2][0] )
SBCStdMinCI.append( DecBCBSCI[2][1][0] )
SBCStdMaxCI.append( DecBCBSCI[2][1][1] )

#### T-Test

T-Test for the means of two independent samples. Two-sided test for the null hypothesis that 2 independent samples hae identical average values. Test assumes that the populations have identical variances.

If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

In [ ]:
DecTTPRtoLO = sstats.ttest_ind( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( DecLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
DecTTPRtoLOPval = DecTTPRtoLO[1]
print("P-Value for PRISM and LOCA from same population is: %g" % DecTTPRtoLOPval )

In [ ]:
DecTTPRtoBC = sstats.ttest_ind( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), 
                                np.array( DecBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
DecTTPRtoBCPval = DecTTPRtoBC[1]
print("P-Value for PRISM and BCCA from same population is: %g" % DecTTPRtoBCPval )

In [ ]:
DecTTLOtoBC = sstats.ttest_ind( np.array( DecLODF["Wet_Count"], dtype=np.int32 ), 
                                np.array( DecBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
DecTTLOtoBCPval = DecTTLOtoBC[1]
print("P-Value for LOCA and BCCA from same population is: %g" % DecTTLOtoBCPval )

#### Mann-Whitney Rank Test

Nonparametric test of the null hypothesis that it is equally likely that a randomly selected value from one sample will be less than or greater than a randomly selected value from a second sample. This test can be used to investigate whether two independent samples were selected from populations having the same distribution.

Null hypothesis, $H_{0}$, is that the distributions of both populations are equal.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of being from the same distribution.

In [ ]:
DecMWUPRtoLO = sstats.mannwhitneyu( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( DecLODF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
DecMWUPRtoLOPval = DecMWUPRtoLO[1]
print("P-Value for PRISM and LOCA from same distribution is: %g" % DecMWUPRtoLOPval )

In [ ]:
DecMWUPRtoBC = sstats.mannwhitneyu( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( DecBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
DecMWUPRtoBCPval = DecMWUPRtoBC[1]
print("P-Value for PRISM and BCCA from same distribution is: %g" % DecMWUPRtoBCPval )

In [ ]:
DecMWULOtoBC = sstats.mannwhitneyu( np.array( DecLODF["Wet_Count"], dtype=np.int32 ), 
                                    np.array( DecBCDF["Wet_Count"], dtype=np.int32 ), 
                                    alternative='two-sided' )

In [ ]:
DecMWULOtoBCPval = DecMWULOtoBC[1]
print("P-Value for LOCA and BCCA from same distribution is: %g" % DecMWULOtoBCPval )

#### Kruskal-Wallis Test

The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs.

If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of the same population median.

In [ ]:
DecKWPRtoLO = sstats.kruskal( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( DecLODF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
DecKWPRtoLOPval = DecKWPRtoLO[1]
print("P-Value for PRISM and LOCA have the same median is: %g" % DecKWPRtoLOPval )

In [ ]:
DecKWPRtoBC = sstats.kruskal( np.array( DecPRDF["Wet_Count"], dtype=np.int32 ), 
                              np.array( DecBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
DecKWPRtoBCPval = DecKWPRtoBC[1]
print("P-Value for PRISM and BCCA have the same median is: %g" % DecKWPRtoBCPval )

In [ ]:
DecKWLOtoBC = sstats.kruskal( np.array( DecLODF["Wet_Count"], dtype=np.int32 ), 
                              np.array( DecBCDF["Wet_Count"], dtype=np.int32 ) )

In [ ]:
DecKWLOtoBCPval = DecKWLOtoBC[1]
print("P-Value for LOCA and BCCA have the same median is: %g" % DecKWLOtoBCPval )

In [ ]:
DecLODF["Wet_Count"].median(), DecBCDF["Wet_Count"].median(), DecPRDF["Wet_Count"].median()

#### Plots

In [ ]:
DecPRQs = DecPRDF["Wet_Count"].quantile(q=nCompQs)
DecLOQs = DecLODF["Wet_Count"].quantile(q=nCompQs)
DecBCQs = DecBCDF["Wet_Count"].quantile(q=nCompQs)

In [ ]:
maxQs = max( DecPRQs.max(), DecLOQs.max(), DecBCQs.max() )
maxQs = maxQs + 2.0
maxQs

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( DecPRQs, DecLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( DecPRQs, DecBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( DecLOQs, DecBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Dec Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, maxQs) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.scatter( DecPRQs, DecLOQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM - LOCA")
ax11.scatter( DecPRQs, DecBCQs, c='xkcd:grass green', marker='o', label="PRISM - BCCA")
ax11.scatter( DecLOQs, DecBCQs, c='xkcd:dark orange', marker='o', label="LOCA - BCCA")
ax11.plot( [0.0, maxQs], [0.0, maxQs], marker=None, linestyle='-', color='xkcd:slate', label="1:1")
ax11.set_title('Dec Comparison Q-Q Plot', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration, 1st (days)', fontsize=14 )
ax11.set_ylabel('Wet Spell Duration, 2nd (days)', fontsize=14)
ax11.set_xlim( (0.0, 10.0) )
ax11.set_ylim( (0.0, 10.0) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(8.0, 8.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( DecPRQs, nCompQs, c='xkcd:royal blue', marker='o', linestyle='-', label="PRISM")
ax11.plot( DecLOQs, nCompQs, c='xkcd:grass green', marker='o', linestyle='-', label="CMIP5 - LOCA")
ax11.plot( DecBCQs, nCompQs, c='xkcd:dark orange', marker='o', linestyle='-', label="CMIP5 - BCCA")
ax11.set_title('Dec Empirical CDF', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, maxQs) )
ax11.set_ylim( (0.0, 1.1) )
ax11.grid( b=True )
ax11.legend(loc='lower right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
testS = np.array( [ x for x in range(1, int(maxQs) + 1, 1)], dtype=np.int32 )

In [ ]:
PRpmf = sstats.nbinom.pmf( testS, PRConvR, PRConvP )
LOpmf = sstats.nbinom.pmf( testS, LOConvR, LOConvP )
BCpmf = sstats.nbinom.pmf( testS, BCConvR, BCConvP )

In [ ]:
width = 0.25
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
rects1 = ax11.bar( testS - (0.75*width), PRpmf, width, color="xkcd:royal blue", label="PRISM" )
rects2 = ax11.bar( testS, LOpmf, width, color="xkcd:grass green", label="LOCA" )
rects3 = ax11.bar( testS + (0.75*width), BCpmf, width, color="xkcd:dark orange", label="BCCA" )
ax11.set_title('Dec Fitted Negative Binomial', fontsize=16 )
ax11.set_xlabel('Wet Spell Duration (days)', fontsize=14 )
ax11.set_ylabel('Probability', fontsize=14)
ax11.set_xlim( (0.0, 15.0) )
ax11.grid( b=True )
ax11.legend(loc='upper right')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.2f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

## Final Collations, Plots, and Outputs

In [ ]:
DataDict = { "Min CI Mean" : np.array( SPRMeanMinCI, dtype=np.float32 ), #0
             "Mean" : np.array( SPRMean, dtype=np.float32 ), #1
             "Max CI Mean" : np.array( SPRMeanMaxCI, dtype=np.float32 ), #2
             "Min CI Var" : np.array( SPRVarMinCI, dtype=np.float32 ), #3
             "Var" : np.array( SPRVar, dtype=np.float32 ), #4
             "Max CI Var" : np.array( SPRVarMaxCI, dtype=np.float32 ), #5
             "Min CI Std" : np.array( SPRStdMinCI, dtype=np.float32 ), #6
             "Std" : np.array( SPRStd, dtype=np.float32 ), #7
             "Max CI Std" : np.array( SPRStdMaxCI, dtype=np.float32 ), #8
}
PRBayStatsCIDF = pd.DataFrame( index=[x for x in range(1, 13, 1)], data=DataDict )

In [ ]:
BSCIHds = list( PRBayStatsCIDF.columns )

In [ ]:
display( HTML( PRBayStatsCIDF.to_html() ) )

In [ ]:
DataDict = { "Min CI Mean" : np.array( SLOMeanMinCI, dtype=np.float32 ), #0
             "Mean" : np.array( SLOMean, dtype=np.float32 ), #1
             "Max CI Mean" : np.array( SLOMeanMaxCI, dtype=np.float32 ), #2
             "Min CI Var" : np.array( SLOVarMinCI, dtype=np.float32 ), #3
             "Var" : np.array( SLOVar, dtype=np.float32 ), #4
             "Max CI Var" : np.array( SLOVarMaxCI, dtype=np.float32 ), #5
             "Min CI Std" : np.array( SLOStdMinCI, dtype=np.float32 ), #6
             "Std" : np.array( SLOStd, dtype=np.float32 ), #7
             "Max CI Std" : np.array( SLOStdMaxCI, dtype=np.float32 ), #8
}
LOBayStatsCIDF = pd.DataFrame( index=[x for x in range(1, 13, 1)], data=DataDict )

In [ ]:
display( HTML( LOBayStatsCIDF.to_html() ) )

In [ ]:
DataDict = { "Min CI Mean" : np.array( SBCMeanMinCI, dtype=np.float32 ), #0
             "Mean" : np.array( SBCMean, dtype=np.float32 ), #1
             "Max CI Mean" : np.array( SBCMeanMaxCI, dtype=np.float32 ), #2
             "Min CI Var" : np.array( SBCVarMinCI, dtype=np.float32 ), #3
             "Var" : np.array( SBCVar, dtype=np.float32 ), #4
             "Max CI Var" : np.array( SBCVarMaxCI, dtype=np.float32 ), #5
             "Min CI Std" : np.array( SBCStdMinCI, dtype=np.float32 ), #6
             "Std" : np.array( SBCStd, dtype=np.float32 ), #7
             "Max CI Std" : np.array( SBCStdMaxCI, dtype=np.float32 ), #8
}
BCBayStatsCIDF = pd.DataFrame( index=[x for x in range(1, 13, 1)], data=DataDict )

In [ ]:
display( HTML( BCBayStatsCIDF.to_html() ) )

In [ ]:
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "WetDayComp_1981-2011.xlsx" ) )
with pd.ExcelWriter( OutFiler ) as writer:
    PRDistDF.to_excel( writer, sheet_name="PRISM", na_rep=str(np.nan), columns=HDRS,
                       index=False )
    PRBayStatsCIDF.to_excel( writer, sheet_name="PRISM_CI_Stats", na_rep=str(np.nan), 
                             columns=BSCIHds, index=True, index_label="Month" )
    LODistDF.to_excel( writer, sheet_name="LOCA", na_rep=str(np.nan), columns=HDRS,
                       index=False )
    LOBayStatsCIDF.to_excel( writer, sheet_name="LOCA_CI_Stats", na_rep=str(np.nan), 
                             columns=BSCIHds, index=True, index_label="Month" )
    BCDistDF.to_excel( writer, sheet_name="BCCA", na_rep=str(np.nan), columns=HDRS,
                       index=False )
    BCBayStatsCIDF.to_excel( writer, sheet_name="BCCA_CI_Stats", na_rep=str(np.nan), 
                             columns=BSCIHds, index=True, index_label="Month" )

Plot the monthly confidence intervals with statistic

In [ ]:
PRMeanCI = np.append( np.stack( [np.array(PRBayStatsCIDF.index, dtype=np.float32 ),
                          np.array(PRBayStatsCIDF[BSCIHds[0]], dtype=np.float32 )],
                          axis=1 ),
                      np.stack( [np.flip(np.array(PRBayStatsCIDF.index, dtype=np.float32 ), 0),
                          np.flip(np.array(PRBayStatsCIDF[BSCIHds[2]], dtype=np.float32 ), 0)],
                          axis=1 ), axis=0 )
PRMeanCI = np.vstack( [PRMeanCI[:,:], PRMeanCI[0,:]] )

In [ ]:
LOMeanCI = np.append( np.stack( [np.array(LOBayStatsCIDF.index, dtype=np.float32 ),
                          np.array(LOBayStatsCIDF[BSCIHds[0]], dtype=np.float32 )],
                          axis=1 ),
                      np.stack( [np.flip(np.array(LOBayStatsCIDF.index, dtype=np.float32 ), 0),
                          np.flip(np.array(LOBayStatsCIDF[BSCIHds[2]], dtype=np.float32 ), 0)],
                          axis=1 ), axis=0 )
LOMeanCI = np.vstack( [LOMeanCI[:,:], LOMeanCI[0,:]] )

In [ ]:
BCMeanCI = np.append( np.stack( [np.array(BCBayStatsCIDF.index, dtype=np.float32 ),
                          np.array(BCBayStatsCIDF[BSCIHds[0]], dtype=np.float32 )],
                          axis=1 ),
                      np.stack( [np.flip(np.array(BCBayStatsCIDF.index, dtype=np.float32 ), 0),
                          np.flip(np.array(BCBayStatsCIDF[BSCIHds[2]], dtype=np.float32 ), 0)],
                          axis=1 ), axis=0 )
BCMeanCI = np.vstack( [BCMeanCI[:,:], BCMeanCI[0,:]] )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( PRBayStatsCIDF.index, PRBayStatsCIDF[BSCIHds[1]], color="xkcd:royal blue", 
           linestyle="-", label="PRISM Mean" )
ax11.fill( PRMeanCI[:,0], PRMeanCI[:,1], color="xkcd:light blue", alpha=0.5, 
           label="PRISM 90% CI")
ax11.plot( LOBayStatsCIDF.index, LOBayStatsCIDF[BSCIHds[1]], color="xkcd:grass green", 
           linestyle="-", label="LOCA Mean" )
ax11.fill( LOMeanCI[:,0], LOMeanCI[:,1], color="xkcd:sage", alpha=0.5, 
           label="LOCA 90% CI")
ax11.plot( BCBayStatsCIDF.index, BCBayStatsCIDF[BSCIHds[1]], color="xkcd:dark orange", 
           linestyle="-", label="BCCA Mean" )
ax11.fill( BCMeanCI[:,0], BCMeanCI[:,1], color="xkcd:light orange", alpha=0.5, 
           label="BCCA 90% CI")
ax11.set_title('Monthly Mean Comparison', fontsize=16 )
ax11.set_xlabel('Month', fontsize=14 )
ax11.set_ylabel('Mean Wet Spell Length (days)', fontsize=14)
ax11.set_xlim( (0.0, 13.0) )
ax11.set_ylim( (0.0, 5.0) )
ax11.grid( b=True )
ax11.legend(loc='upper left')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )

In [ ]:
PRVarCI = np.append( np.stack( [np.array(PRBayStatsCIDF.index, dtype=np.float32 ),
                          np.array(PRBayStatsCIDF[BSCIHds[3]], dtype=np.float32 )],
                          axis=1 ),
                      np.stack( [np.flip(np.array(PRBayStatsCIDF.index, dtype=np.float32 ), 0),
                          np.flip(np.array(PRBayStatsCIDF[BSCIHds[5]], dtype=np.float32 ), 0)],
                          axis=1 ), axis=0 )
PRVarCI = np.vstack( [PRVarCI[:,:], PRVarCI[0,:]] )

In [ ]:
LOVarCI = np.append( np.stack( [np.array(LOBayStatsCIDF.index, dtype=np.float32 ),
                          np.array(LOBayStatsCIDF[BSCIHds[3]], dtype=np.float32 )],
                          axis=1 ),
                      np.stack( [np.flip(np.array(LOBayStatsCIDF.index, dtype=np.float32 ), 0),
                          np.flip(np.array(LOBayStatsCIDF[BSCIHds[5]], dtype=np.float32 ), 0)],
                          axis=1 ), axis=0 )
LOVarCI = np.vstack( [LOVarCI[:,:], LOVarCI[0,:]] )

In [ ]:
BCVarCI = np.append( np.stack( [np.array(BCBayStatsCIDF.index, dtype=np.float32 ),
                          np.array(BCBayStatsCIDF[BSCIHds[3]], dtype=np.float32 )],
                          axis=1 ),
                      np.stack( [np.flip(np.array(BCBayStatsCIDF.index, dtype=np.float32 ), 0),
                          np.flip(np.array(BCBayStatsCIDF[BSCIHds[5]], dtype=np.float32 ), 0)],
                          axis=1 ), axis=0 )
BCVarCI = np.vstack( [BCVarCI[:,:], BCVarCI[0,:]] )

In [ ]:
Fig1 = plt.figure()
Fig1.set_size_inches(10.0, 6.0)
ax11 = Fig1.add_subplot(111)
ax11.plot( PRBayStatsCIDF.index, PRBayStatsCIDF[BSCIHds[4]], color="xkcd:royal blue", 
           linestyle="-", label="PRISM Var" )
ax11.fill( PRVarCI[:,0], PRVarCI[:,1], color="xkcd:light blue", alpha=0.5, 
           label="PRISM 90% CI")
ax11.plot( LOBayStatsCIDF.index, LOBayStatsCIDF[BSCIHds[4]], color="xkcd:grass green", 
           linestyle="-", label="LOCA Var" )
ax11.fill( LOVarCI[:,0], LOVarCI[:,1], color="xkcd:sage", alpha=0.5, 
           label="LOCA 90% CI")
ax11.plot( BCBayStatsCIDF.index, BCBayStatsCIDF[BSCIHds[4]], color="xkcd:dark orange", 
           linestyle="-", label="BCCA Var" )
ax11.fill( BCVarCI[:,0], BCVarCI[:,1], color="xkcd:light orange", alpha=0.5, 
           label="BCCA 90% CI")
ax11.set_title('Monthly Variance Comparison', fontsize=16 )
ax11.set_xlabel('Month', fontsize=14 )
ax11.set_ylabel('Variance of Wet Spell Length ($days^{2}$)', fontsize=14)
ax11.set_xlim( (0.0, 13.0) )
ax11.set_ylim( (0.0, 12.0) )
ax11.grid( b=True )
ax11.legend(loc='upper left')
ax11.tick_params(axis='both', which='major', labelsize=13)
ax11.yaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.1f}" ) )
ax11.xaxis.set_major_formatter( mpl.ticker.StrMethodFormatter( "{x:,.0f}" ) )